In [ ]:
import numpy as np
import pandas as pd
import requests
import random
import time
from io import StringIO


In [ ]:
teams = [
    'ATL', 'BOS', 'BRK', 'CHO', 'CHI', 'DAL', 'DEN', 'DET', 'GSW', 'HOU',
    'CLE', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK',
    'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS'
]


In [ ]:
team_dict ={
    'ATL': 'Atlanta Hawks',
    'BOS': 'Boston Celtics',
    'BRK': 'Brooklyn Nets',
    'CHO': 'Charlotte Hornets',
    'CHI': 'Chicago Bulls',
    'CLE': 'Cleveland Cavaliers',
    'DAL': 'Dallas Mavericks',
    'DEN': 'Denver Nuggets',
    'DET': 'Detroit Pistons',
    'GSW': 'Golden State Warriors',
    'HOU': 'Houston Rockets',
    'IND': 'Indiana Pacers',
    'LAC': 'Los Angeles Clippers',
    'LAL': 'Los Angeles Lakers',
    'MEM': 'Memphis Grizzlies',
    'MIA': 'Miami Heat',
    'MIL': 'Milwaukee Bucks',
    'MIN': 'Minnesota Timberwolves',
    'NOP': 'New Orleans Pelicans',
    'NYK': 'New York Knicks',
    'OKC': 'Oklahoma City Thunder',
    'ORL': 'Orlando Magic',
    'PHI': 'Philadelphia 76ers',
    'PHO': 'Phoenix Suns',
    'POR': 'Portland Trail Blazers',
    'SAC': 'Sacramento Kings',
    'SAS': 'San Antonio Spurs',
    'TOR': 'Toronto Raptors',
    'UTA': 'Utah Jazz',
    'WAS': 'Washington Wizards'
}

In [ ]:
seasons =['2019','2020','2021','2022','2023','2024']
#len(seasons)
#print(seasons)

for season in seasons:
  print(season)

2019
2020
2021
2022
2023
2024


In [ ]:
# Create an empty DataFrame to append
nba_df = pd.DataFrame()

# Iterate through seasons and teams
for season in seasons:
    for team in teams:

        # Set up URL
        url = f'https://www.basketball-reference.com/teams/{team}/{season}/gamelog-advanced/'
        print(url)

        response = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
        visible_html = response.text.replace('<!--', '').replace('-->', '')

        try:
            df_list = pd.read_html(StringIO(visible_html), header=1)
            if len(df_list) > 1:
                # If playoffs exist, concatenate regular season and playoff data
                df_list[1]['G'] = df_list[1]['G'] + 82  # Offset playoff games
                team_df = pd.concat([df_list[0], df_list[1]], ignore_index=True)
            else:
                team_df = df_list[0]

            # Clean and process DataFrame
            team_df = team_df.dropna(subset=['G'])  # Drop rows where 'G' is NaN
            team_df = team_df[team_df['G'] != 'G']  # Remove header rows
            team_df = team_df.iloc[:, 1:11]  # Keep columns 1 to 11
            team_df = team_df.rename(columns={'Unnamed: 3': 'At', 'Tm': 'OPts', 'Opp.1': 'DPts'})

            # Add computed columns
            team_df['At'] = team_df['At'].apply(lambda x: x if x == '@' else "")
            team_df['Win'] = team_df['W/L'].apply(lambda x: 1 if x == 'W' else 0)
            team_df['Location'] = team_df['At'].apply(lambda x: 'Away' if x == '@' else 'Home')
            team_df.insert(loc =6 , column= 'Location', value=team_df.pop('Location'))
            team_df['Opp_Name'] = team_df['Opp'].apply(lambda x: team_dict.get(x, 'Unknown'))
            team_df.insert(loc =8 , column= 'Opp_Name', value=team_df.pop('Opp_Name'))

            # Insert additional columns
            team_df.insert(0, 'Season', season)
            team_df.insert(2, 'Team', team)
            team_df.insert(3, 'Name', team_dict[team])

            # Append to aggregate DataFrame
            nba_df = pd.concat([nba_df, team_df], ignore_index=True)

        except Exception as e:
            print(f"Failed to process {team} in {season}: {e}")

        # Pause to abide by site rules
        time.sleep(random.randint(4, 6))

# Display the aggregate DataFrame
print(nba_df.shape)


https://www.basketball-reference.com/teams/ATL/2019/gamelog-advanced/
https://www.basketball-reference.com/teams/BOS/2019/gamelog-advanced/
https://www.basketball-reference.com/teams/BRK/2019/gamelog-advanced/
https://www.basketball-reference.com/teams/CHO/2019/gamelog-advanced/
https://www.basketball-reference.com/teams/CHI/2019/gamelog-advanced/
https://www.basketball-reference.com/teams/DAL/2019/gamelog-advanced/
https://www.basketball-reference.com/teams/DEN/2019/gamelog-advanced/
https://www.basketball-reference.com/teams/DET/2019/gamelog-advanced/
https://www.basketball-reference.com/teams/GSW/2019/gamelog-advanced/
Failed to process GSW in 2019: can only concatenate str (not "int") to str
https://www.basketball-reference.com/teams/HOU/2019/gamelog-advanced/
https://www.basketball-reference.com/teams/CLE/2019/gamelog-advanced/
https://www.basketball-reference.com/teams/IND/2019/gamelog-advanced/
https://www.basketball-reference.com/teams/LAC/2019/gamelog-advanced/
https://www.bas

In [ ]:
nba_df

,Season,G,Team,Name,Date,At,Opp,W/L,OPts,Location,DPts,Opp_Name,ORtg,DRtg,Pace,Win
0,2019,1,ATL,Atlanta Hawks,2018-10-17,@,NYK,L,107,Away,126,New York Knicks,93.8,110.5,114.0,0
1,2019,2,ATL,Atlanta Hawks,2018-10-19,@,MEM,L,117,Away,131,Memphis Grizzlies,109.9,123.1,106.5,0
2,2019,3,ATL,Atlanta Hawks,2018-10-21,@,CLE,W,133,Away,111,Cleveland Cavaliers,125.5,104.7,106.0,1
3,2019,4,ATL,Atlanta Hawks,2018-10-24,,DAL,W,111,Home,104,Dallas Mavericks,102.7,96.2,108.1,1
4,2019,5,ATL,Atlanta Hawks,2018-10-27,,CHI,L,85,Home,97,Chicago Bulls,82.3,94.0,103.2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14115,2024,78,WAS,Washington Wizards,2024-04-05,,POR,L,102,Home,108,Portland Trail Blazers,104.4,110.5,97.7,0
14116,2024,79,WAS,Washington Wizards,2024-04-07,@,TOR,L,122,Away,130,Toronto Raptors,111.1,118.4,109.8,0
14117,2024,80,WAS,Washington Wizards,2024-04-09,@,MIN,L,121,Away,130,Minnesota Timberwolves,121.0,130.0,100.0,0
14118,2024,81,WAS,Washington Wizards,2024-04-12,,CHI,L,127,Home,129,Chicago Bulls,120.8,122.7,105.1,0


In [ ]:
# Update datatypes
nba_df['G'] = nba_df['G'].astype('int')
nba_df['OPts'] = nba_df['OPts'].astype('int')
nba_df['DPts'] = nba_df['DPts'].astype('int')
nba_df['ORtg'] = nba_df['ORtg'].astype('float')
nba_df['DRtg'] = nba_df['DRtg'].astype('float')
nba_df['Pace'] = nba_df['Pace'].astype('float')

In [ ]:
print(nba_df.info())
(nba_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14120 entries, 0 to 14119
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Season    14120 non-null  object 
 1   G         14120 non-null  int64  
 2   Team      14120 non-null  object 
 3   Name      14120 non-null  object 
 4   Date      14120 non-null  object 
 5   At        14120 non-null  object 
 6   Opp       14120 non-null  object 
 7   W/L       14120 non-null  object 
 8   OPts      14120 non-null  int64  
 9   Location  14120 non-null  object 
 10  DPts      14120 non-null  int64  
 11  Opp_Name  14120 non-null  object 
 12  ORtg      14120 non-null  float64
 13  DRtg      14120 non-null  float64
 14  Pace      14120 non-null  float64
 15  Win       14120 non-null  int64  
dtypes: float64(3), int64(4), object(9)
memory usage: 1.7+ MB
None


,Season,G,Team,Name,Date,At,Opp,W/L,OPts,Location,DPts,Opp_Name,ORtg,DRtg,Pace,Win
0,2019,1,ATL,Atlanta Hawks,2018-10-17,@,NYK,L,107,Away,126,New York Knicks,93.8,110.5,114.0,0
1,2019,2,ATL,Atlanta Hawks,2018-10-19,@,MEM,L,117,Away,131,Memphis Grizzlies,109.9,123.1,106.5,0
2,2019,3,ATL,Atlanta Hawks,2018-10-21,@,CLE,W,133,Away,111,Cleveland Cavaliers,125.5,104.7,106.0,1
3,2019,4,ATL,Atlanta Hawks,2018-10-24,,DAL,W,111,Home,104,Dallas Mavericks,102.7,96.2,108.1,1
4,2019,5,ATL,Atlanta Hawks,2018-10-27,,CHI,L,85,Home,97,Chicago Bulls,82.3,94.0,103.2,0


In [ ]:
nba_df.to_csv('nba_advanced_2019-2024_data.csv', index=False)